In [1]:
import pandas as pd
from pathlib import Path
from itertools import combinations

In [21]:
airport_data_path = "/home/danhle/AIATFM/data_preparation/deepflightplan/datasets/airports/iata-icao.csv"

In [22]:
data_dir = Path("/home/shared/data/oag/OAG_Jan1toJan152024").glob("*Jan152024*")
df = pd.DataFrame()
for file in data_dir:
    if file.is_file():
        print(file)
        tmp = pd.read_csv(file)
        df = pd.concat([df, tmp], ignore_index=True)

df = df.sort_values(by=['Time series'])
df.reset_index(drop=True, inplace=True)
print("before drop duplicate df: ", len(df))
df.drop_duplicates(inplace=True)
print("after drop duplicate df:",len(df))

/home/shared/data/oag/OAG_Jan1toJan152024/EU_Jan01Jan152024_JobId3284708.csv
/home/shared/data/oag/OAG_Jan1toJan152024/AfricaJan01Jan152024_JobId3284838.csv
/home/shared/data/oag/OAG_Jan1toJan152024/NA_Jan01toJan152024_JobId3284834.csv
/home/shared/data/oag/OAG_Jan1toJan152024/LA_Jan01Jan152024_JobId3284839.csv
/home/shared/data/oag/OAG_Jan1toJan152024/ME_Jan01toJan152024_JobId3284835.csv
/home/shared/data/oag/OAG_Jan1toJan152024/SW_Jan01toJan152024_JobId3284714.csv
before drop duplicate df:  1090727
after drop duplicate df: 977919


In [23]:
df1 = df.copy()

df2 = pd.read_csv('/home/shared/data/oag/2024jan1_15_asean.zip', compression='zip')
df1.drop(columns=list(set(df1.columns) - set(df2.columns)), inplace=True)

print("srinivas method: ", len(df1))
print("Minh's data: ", len(df2))

srinivas method:  977919
Minh's data:  1413601


In [24]:
df3 = pd.read_csv('/home/shared/data/oag/202401.zip', compression='zip')
df3.columns

Index(['Carrier Code', 'Dep Airport Code', 'Dep Airport Name', 'Dep City Name',
       'Dep DOT Country Name', 'Dep Region Name', 'Arr City Name',
       'Arr Airport Code', 'Arr Airport Name', 'Arr DOT Country Name',
       'Arr Region Name', 'International/Domestic', 'Flying Time',
       'Elapsed Time', 'Local Dep Time', 'Local Arr Time',
       'Specific Aircraft Code', 'General Aircraft Code',
       'Aircraft Range (km)', 'Seats', 'GCD (km)', 'GCD (NM)', 'Routing',
       'Aircraft Cruise Speed (MPH)', 'Aircraft Hold Volume (M3)', 'Flight No',
       'Carrier Name', 'FlyingTime (Minutes)', 'Economy seats (Total)',
       'Business seats (Total)', 'Seats (Total)', 'Time series'],
      dtype='object')

In [25]:
def filter_df_by_time_range(df, start_time="2024-01-01", end_time="2024-01-01"):
    df['Time series'] = pd.to_datetime(df['Time series'], format="%Y-%m-%d")
    mask = (df['Time series'] >= start_time) & (df['Time series'] <= end_time)
    return df.loc[mask]


df1 = filter_df_by_time_range(df1, start_time="2024-01-01", end_time="2024-01-15")
df2 = filter_df_by_time_range(df2, start_time="2024-01-01", end_time="2024-01-15")
df3 = filter_df_by_time_range(df3, start_time="2024-01-01", end_time="2024-01-15")

In [26]:
df1.columns = sorted(df1.columns)
df2.columns = sorted(df2.columns)
df3.columns = sorted(df3.columns)

intersection_columns = list(set(df1.columns) & set(df2.columns) & set(df3.columns))
df1 = df1[intersection_columns]
df2 = df2[intersection_columns]
df3 = df3[intersection_columns]

In [30]:
# Union of two dataframes
# Convert dataframes to sets of tuples

def union_rows(df1,df2):
    rows_df1 = set(df1.itertuples(index=False, name=None))
    rows_df2 = set(df2.itertuples(index=False, name=None))

    # Perform union
    union_rows = rows_df1 | rows_df2
    union_df = pd.DataFrame(list(union_rows), columns=df1.columns)
    return union_df


In [31]:
def intersection_rows(df1,df2):
    rows_df1 = set(df1.itertuples(index=False, name=None))
    rows_df2 = set(df2.itertuples(index=False, name=None))

    # Perform intersection
    intersection_rows = rows_df1 & rows_df2
    intersection_df = pd.DataFrame(list(intersection_rows), columns=df1.columns)
    return intersection_df

In [32]:
def subtract_rows(df1,df2):
    rows_df1 = set(df1.itertuples(index=False, name=None))
    rows_df2 = set(df2.itertuples(index=False, name=None))

    # Perform subtraction
    subtract_rows = rows_df1 - rows_df2
    subtract_df = pd.DataFrame(list(subtract_rows), columns=df1.columns)
    return subtract_df

In [54]:
df_danh = pd.read_csv("/home/danhle/AIATFM/AS_Jan012024_JobId3290812.csv")
df_danh_no_codeshare = pd.read_csv("/home/danhle/AIATFM/AS_Jan012024_nocodsh_JobId3290816.csv")
df_danh_simple = pd.read_csv("/home/danhle/AIATFM/AS_Jun012024_Simple_JobId3290891.csv")
intersection_columns = list(set(df_danh.columns) & set(df_danh_no_codeshare.columns) & set(df1.columns) & set(df2.columns) & set(df3.columns) & set(df_danh_simple.columns))

df_danh = df_danh[intersection_columns]
df_danh_no_codeshare = df_danh_no_codeshare[intersection_columns]
df_danh_simple = df_danh_simple[intersection_columns]
df_danh['Time series'] = pd.to_datetime(df_danh['Time series'], format="%Y-%m-%d")
df_danh_no_codeshare['Time series'] = pd.to_datetime(df_danh_no_codeshare['Time series'], format="%Y-%m-%d")
df_danh["Time series"] = df_danh["Time series"].dt.strftime("%Y-%m-%d")

df_danh.columns = sorted(df_danh.columns)
df_danh_no_codeshare.columns = sorted(df_danh_no_codeshare.columns)

intersection_df = intersection_rows(df_danh, df_danh_no_codeshare)
# print("Intersection of df_danh and df_danh_no_codeshare: ", len(intersection_df))
# print("Substraction of df_danh and df_danh_no_codeshare: ", len(subtract_rows(df_danh, df_danh_no_codeshare)))
# print("Subtraction of df_danh_no_codeshare and df_danh: ", len(subtract_rows(df_danh_no_codeshare, df_danh)))


asia_region = ['Asia : North East Asia', 'Asia : South Asia', 'Asia : South East Asia', 'Asia : Central Asia']

# compare with df1, df2, df3
# tmp_df1 = filter_df_by_time_range(df1, start_time="2024-01-01", end_time="2024-01-01")
# tmp_df2 = filter_df_by_time_range(df2, start_time="2024-01-01", end_time="2024-01-01")
# tmp_df3 = filter_df_by_time_range(df3, start_time="2024-01-01", end_time="2024-01-01")

# Filter region asia from df1, df2, df3
# tmp_df1 = tmp_df1[tmp_df1['Arr Region Name'].isin(asia_region) | tmp_df1['Dep Region Name'].isin(asia_region)]
# tmp_df2 = tmp_df2[tmp_df2['Arr Region Name'].isin(asia_region) | tmp_df2['Dep Region Name'].isin(asia_region)]
# tmp_df3 = tmp_df3[tmp_df3['Arr Region Name'].isin(asia_region) | tmp_df3['Dep Region Name'].isin(asia_region)]


list_of_dfs_and_names = [
    (tmp_df1, "srinivas"),
    (tmp_df2, "minh"),
    (tmp_df3, "trunghoang"),
    (df_danh, "df_danh"),
    (df_danh_no_codeshare, "df_danh_no_codeshare"),
    (df_danh_simple, "danh_simple"),
]

for (df1, name1), (df2, name2) in combinations(list_of_dfs_and_names, 2):
    # Perform union
    # union_df = union_rows(df1, df2)
    # print(f"Union of {name1} and {name2}: ", len(union_df))

    # Perform intersection
    intersection_df = intersection_rows(df1, df2)
    print(f"Intersection of {name1} and {name2}: ", len(intersection_df), f"{name1}: ", len(df1), f"{name2}: ", len(df2))

    # Perform subtraction
    # subtract_df = subtract_rows(df1, df2)
    # print(f"Subtraction of {name1} and {name2}: ", len(subtract_df))




Intersection of srinivas and minh:  0 srinivas:  1561 minh:  30295
Intersection of srinivas and trunghoang:  0 srinivas:  1561 trunghoang:  0
Intersection of srinivas and df_danh:  0 srinivas:  1561 df_danh:  51531
Intersection of srinivas and df_danh_no_codeshare:  0 srinivas:  1561 df_danh_no_codeshare:  31915
Intersection of srinivas and danh_simple:  0 srinivas:  1561 danh_simple:  51585
Intersection of minh and trunghoang:  0 minh:  30295 trunghoang:  0
Intersection of minh and df_danh:  0 minh:  30295 df_danh:  51531
Intersection of minh and df_danh_no_codeshare:  0 minh:  30295 df_danh_no_codeshare:  31915
Intersection of minh and danh_simple:  0 minh:  30295 danh_simple:  51585
Intersection of trunghoang and df_danh:  0 trunghoang:  0 df_danh:  51531
Intersection of trunghoang and df_danh_no_codeshare:  0 trunghoang:  0 df_danh_no_codeshare:  31915
Intersection of trunghoang and danh_simple:  0 trunghoang:  0 danh_simple:  51585
Intersection of df_danh and df_danh_no_codeshare:

In [48]:
len(df_danh_no_codeshare), len(df_danh)

(31915, 51531)

In [33]:
# Perform union, intersection, and subtraction of df1 and df2
union_df = union_rows(df1, df2)
intersection_df = intersection_rows(df1, df2)
print("Union of Srinivas and Minh: ", len(union_df))
print("Intersection of Srinivas and Minh: ", len(intersection_df))
print("Subtract of Srinivas and Minh: ", len(subtract_rows(df1, df2))) 
print("Subtract of Minh and Srinivas: ", len(subtract_rows(df2, df1)))

Union of Srinivas and Minh:  2391520
Intersection of Srinivas and Minh:  0
Subtract of Srinivas and Minh:  977919
Subtract of Minh and Srinivas:  1413601


In [34]:
# Perform union, intersection, and subtraction of df2 and df3
union_df = union_rows(df2, df3)
intersection_df = intersection_rows(df2, df3)
print("Union of Minh and Trung-Hoang: ", len(union_df))
print("Intersection of Minh and Trung-Hoang: ", len(intersection_df))
print("Subtract of Minh and Trung-Hoang: ", len(subtract_rows(df2, df3))) 
print("Subtract of Trung-Hoang and Minh: ", len(subtract_rows(df3, df2)))

Union of Minh and Trung-Hoang:  2168498
Intersection of Minh and Trung-Hoang:  0
Subtract of Minh and Trung-Hoang:  1413601
Subtract of Trung-Hoang and Minh:  754897


In [35]:
# Perform union, intersection, and subtraction of df1 and df3
union_df = union_rows(df1, df3)
intersection_df = intersection_rows(df1, df3)
print("Union of Srinivas and Trung-Hoang: ", len(union_df))
print("Intersection of Srinivas and Trung-Hoang: ", len(intersection_df))
print("Subtract of Srinivas and Trung-Hoang: ", len(subtract_rows(df1, df3))) 
print("Subtract of Trung-Hoang and Srinivas: ", len(subtract_rows(df3, df1)))

Union of Srinivas and Trung-Hoang:  1732816
Intersection of Srinivas and Trung-Hoang:  0
Subtract of Srinivas and Trung-Hoang:  977919
Subtract of Trung-Hoang and Srinivas:  754897
